In [ ]:
import time
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as tkr
from datetime import datetime, timedelta

from cubingpa import (
    utils,
    data_filter,
    db_data_loader)
from cubingpa.events import EventId
from cubingpa.reference_processor import ReferenceProcessor

In [ ]:
total_start_time = time.time()
event = EventId.E_555_BF

In [ ]:
raw_data = db_data_loader.load()
filtered_results = data_filter.filter(raw_data, event)

In [ ]:
processor = ReferenceProcessor(filtered_results)

In [ ]:
processed_results = processor.process(True)

In [ ]:
# very long
#general_data.final_df.plot(figsize=[16, 4.5])

In [ ]:
final_average_df = pd.DataFrame(processed_results.mean(axis=1), columns=['Average time'])

In [ ]:
def format_time(x, pos):
    return str(timedelta(seconds=x))

time_formatter = tkr.FuncFormatter(format_time)

In [ ]:
def format_timedelta(x, pos):
    # x is in nanoseconds
    date = datetime(2001, 1, 1) + timedelta(microseconds=x/1000)
    d = {'years': (date.year - 2001), 'months': (date.month - 1), 'days': (date.day - 1)}
    return "{years}y {months}m {days}d".format(**d)

timedelta_formatter = tkr.FuncFormatter(format_timedelta)

In [ ]:
final_average_df = utils.remove_not_progressing_solves(final_average_df)
# with timedelta, if index is not interpolated axis is messed up when plotting 
final_average_df = utils.interpolate_dates(final_average_df)
final_average_df = utils.convert_date_index_to_timedelta(final_average_df)

progressing_average_timedelta_graph = final_average_df.plot(figsize=[16, 4.5],
                                                            title=event.value + " progressing timedelta average")
progressing_average_timedelta_graph.yaxis.set_major_formatter(time_formatter)
progressing_average_timedelta_graph.xaxis.set_major_formatter(timedelta_formatter)

In [ ]:
for row in final_average_df.itertuples():
    print(row[0], row[1])

In [ ]:
print(f'Total time: {time.time() - total_start_time} seconds')